#### Installing Dependencies

In [ ]:
!pip install torchmetrics

In [ ]:
!sudo apt install octave

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
octave is already the newest version (6.4.0-2).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
!pip install oct2py

#### Importing Pkgs

In [ ]:
from oct2py import Oct2Py

oc = Oct2Py();

In [ ]:
import torch;
import torch.nn as nn;
import torch.nn.functional as F;
from torch.utils.data import Dataset, DataLoader;

import matplotlib.pyplot as plt;
import numpy as np;

from tqdm import tqdm;

In [ ]:
from torchmetrics.classification import Accuracy,Precision, Recall, F1Score, ConfusionMatrix;

In [ ]:
import pandas as pd;
import random;

In [ ]:
import json;

In [ ]:
import os;
import os.path;

In [ ]:
import re;

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu");
print(device);

cuda


#### Dataset Creation

In [ ]:
seed = 42;

In [ ]:
np.random.seed(seed);
random.seed(seed);
torch.manual_seed(seed);

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
    torch.cuda.manual_seed_all(seed);

In [ ]:
def getYLabel(filename):
    output = {
            "K" : 0,
            "KA" : 1,
            "KI" : 2
    };

    code = filename.split("_")[3];

    if code.startswith("KA"):
        return output["KA"];
    elif code.startswith("KI"):
        return output["KI"];
    else:
        return output["K"];

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, code, bearing_end_points, oc, size, signal="vibration"):
        self.root_dir = root_dir;

        self.X = [];
        self.y = [];

        self.code = code;
        self.bearing_end_points = bearing_end_points;

        self.oc = oc;

        self.size = size;

        fieldToIndex = {
        "force" : 0,
        "current_1" : 1,
        "current_2" : 2,
        "speed" : 3,
        "temp" : 4,
        "torque" : 5,
        "vibration" : 6
        };

        for bearing_end_point in bearing_end_points:
            for mat_bearing_end_point in os.listdir(os.path.join(self.root_dir, bearing_end_point)):
                if mat_bearing_end_point.endswith(".mat") and mat_bearing_end_point.startswith(code):
                    data = self.oc.load(os.path.join(self.root_dir, bearing_end_point, mat_bearing_end_point));
                    field = list(data.keys())[0];

                    x = data[field]['Y']['Data'][0][fieldToIndex[signal]].flatten();

                    chunk_size = 2500;

                    y = getYLabel(mat_bearing_end_point);

                    for i in range(0, x.shape[-1]-chunk_size, chunk_size):
                        self.X.append(x[i:i+chunk_size]);
                        self.y.append(y);

    def __len__(self):
        return len(self.X);

    def __getitem__(self, idx):
        x = self.X[idx];
        y = self.y[idx];

        x = torch.tensor(x, dtype=torch.float32).reshape((-1,*self.size));
        y = torch.tensor(y, dtype=torch.long);

        return x, y;

In [ ]:
# ratios = {2:5, 6:5, 8:5}
def getDataset(code, signal = "vibration", size=(50,50), ratios="6:5"):
    root_dir = "/content/drive/MyDrive/PaderbornExtracted";
    healty_end_points = ["K001","K002","K003","K004","K005","K006"];
    faulty_end_points_inner = ["KI04","KI14","KI16","KI17","KI18","KI21"];
    faulty_end_points_outer = ["KA04","KA15","KA16","KA22","KA30"];

    f,h = ratios.split(":");

    f,h = int(f), int(h);

    training_end_points = [];
    testing_end_points = [];


    training_end_points += healty_end_points[:h] + faulty_end_points_inner[:f//2] + faulty_end_points_outer[:f//2];
    testing_end_points += healty_end_points[h:] + faulty_end_points_inner[f//2:] + faulty_end_points_outer[f//2:];

    train_dataset = CustomDataset(root_dir, code, training_end_points, oc, size, signal);
    test_dataset = CustomDataset(root_dir, code, testing_end_points, oc, size, signal);

    return train_dataset, test_dataset;

In [ ]:
def getCode(N, M, F):
    return f"N{N:02}" + "_" + f"M{M:02}" + "_" + f"F{F:02}";

code = getCode(15, 7, 10);

In [ ]:
training_dataset, testing_dataset = getDataset(code,signal="vibration");

In [ ]:
training_dataloader = DataLoader(training_dataset, batch_size=128, shuffle=True);
testing_dataloader = DataLoader(testing_dataset, batch_size=128, shuffle=True);

#### Model Creation

In [ ]:
class FaultNet(nn.Module):
    def __init__(self):
        super(FaultNet, self).__init__();

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(4,4), stride=(1,1) );
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(4,4), stride=(1,1));
        self.pool = nn.MaxPool2d(2, 2);

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(4,4), stride=(1,1));

        self.fc1 = nn.Linear(2592, 256);
        self.fc2 = nn.Linear(256, 3);

    def forward(self, x):
        x = F.relu(self.conv1(x));
        x = F.relu(self.conv2(x));
        x = self.pool(x);
        x = F.relu(self.conv3(x));
        x = self.pool(x);
        x = x.view(-1, 2592);
        x = F.relu(self.fc1(x));
        x = self.fc2(x);

        return x;


In [ ]:
model = FaultNet().to(device);

print(model);

FaultNet(
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 32, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=2592, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=3, bias=True)
)


In [ ]:
loss = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001);

In [ ]:
accuracy = Accuracy(task="multiclass", num_classes=3).to(device);
precision = Precision(task="multiclass", num_classes=3).to(device);
recall = Recall(task="multiclass", num_classes=3).to(device);
f1_score = F1Score(task="multiclass", num_classes=3).to(device);
conf_matrix_metric = ConfusionMatrix(task="multiclass", num_classes=3).to(device);

In [ ]:
result_dir = "/content/drive/MyDrive/Results/CNN-split-3";
if not os.path.exists(result_dir):
    os.makedirs(result_dir);

#### Model Training

In [ ]:
epochs = 75;

In [ ]:
metrics_list = [];

for epoch in range(epochs):

    train_loss_per_epoch = [];
    test_loss_per_epcoh = [];

    train_acc_per_epoch = [];
    test_acc_per_epoch = [];

    train_precision_per_epoch = [];
    test_precision_per_epoch = [];

    train_recall_per_epoch = [];
    test_recall_per_epoch = [];

    train_f1_per_epoch = [];
    test_f1_per_epoch = [];

    traing_cf = np.zeros((3,3));
    test_cf = np.zeros((3,3));

    model.train();
    for x, y in training_dataloader:
        optimizer.zero_grad();
        x = x.to(device);
        y = y.to(device);

        y_pred = model(x);
        loss_value = loss(y_pred, y);

        loss_value.backward();
        optimizer.step();

        acc = accuracy(y_pred, y);
        prec = precision(y_pred, y);
        rec = recall(y_pred, y);
        f1 = f1_score(y_pred, y);
        conf_matrix = conf_matrix_metric(y_pred, y);

        traing_cf += conf_matrix.cpu().numpy();

        train_loss_per_epoch.append(loss_value.item());
        train_acc_per_epoch.append(acc.item());
        train_precision_per_epoch.append(prec.item());
        train_recall_per_epoch.append(rec.item());
        train_f1_per_epoch.append(f1.item());

    model.eval();
    with torch.no_grad():
        for x, y in testing_dataloader:
            x = x.to(device);
            y = y.to(device);

            y_pred = model(x);
            loss_value = loss(y_pred, y);

            acc = accuracy(y_pred, y);
            prec = precision(y_pred, y);
            rec = recall(y_pred, y);
            f1 = f1_score(y_pred, y);
            conf_matrix = conf_matrix_metric(y_pred, y);

            test_cf += conf_matrix.cpu().numpy();

            test_loss_per_epcoh.append(loss_value.item());
            test_acc_per_epoch.append(acc.item());
            test_precision_per_epoch.append(prec.item());
            test_recall_per_epoch.append(rec.item());
            test_f1_per_epoch.append(f1.item());


    metrics_list.append({
        "train_loss": np.mean(train_loss_per_epoch),
        "test_loss": np.mean(test_loss_per_epcoh),
        "train_acc": np.mean(train_acc_per_epoch),
        "test_acc": np.mean(test_acc_per_epoch),
        "train_precision": np.mean(train_precision_per_epoch),
        "test_precision": np.mean(test_precision_per_epoch),
        "train_recall": np.mean(train_recall_per_epoch),
        "test_recall": np.mean(test_recall_per_epoch),
        "train_f1": np.mean(train_f1_per_epoch),
        "test_f1": np.mean(test_f1_per_epoch),
        "train_cf": json.dumps(traing_cf.tolist()),
        "test_cf": json.dumps(test_cf.tolist())
    });

    maxx = max(metrics_list, key=lambda x: x["test_f1"]);

    if maxx["test_f1"] <= metrics_list[-1]["test_f1"]:
        torch.save(model.state_dict(), os.path.join(result_dir,"bb_mm.pth"));

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {metrics_list[-1]['train_loss']:.4f} | Test Loss: {metrics_list[-1]['test_loss']:.4f} | Train Acc: {metrics_list[-1]['train_acc']:.4f} | Test Acc: {metrics_list[-1]['test_acc']:.4f} | Train F1: {metrics_list[-1]['train_f1']:.4f} | Test F1: {metrics_list[-1]['test_f1']:.4f}");

Epoch 1/75 | Train Loss: 0.2753 | Test Loss: 1.7803 | Train Acc: 0.8676 | Test Acc: 0.7093 | Train F1: 0.8676 | Test F1: 0.7093
Epoch 2/75 | Train Loss: 0.0518 | Test Loss: 2.2682 | Train Acc: 0.9808 | Test Acc: 0.7631 | Train F1: 0.9808 | Test F1: 0.7631
Epoch 3/75 | Train Loss: 0.0184 | Test Loss: 2.5660 | Train Acc: 0.9935 | Test Acc: 0.7589 | Train F1: 0.9935 | Test F1: 0.7589
Epoch 4/75 | Train Loss: 0.0078 | Test Loss: 3.0137 | Train Acc: 0.9974 | Test Acc: 0.7834 | Train F1: 0.9974 | Test F1: 0.7834
Epoch 5/75 | Train Loss: 0.0167 | Test Loss: 1.9631 | Train Acc: 0.9945 | Test Acc: 0.7665 | Train F1: 0.9945 | Test F1: 0.7665
Epoch 6/75 | Train Loss: 0.0083 | Test Loss: 3.3231 | Train Acc: 0.9968 | Test Acc: 0.6909 | Train F1: 0.9968 | Test F1: 0.6909
Epoch 7/75 | Train Loss: 0.0089 | Test Loss: 2.9785 | Train Acc: 0.9973 | Test Acc: 0.7746 | Train F1: 0.9973 | Test F1: 0.7746
Epoch 8/75 | Train Loss: 0.0260 | Test Loss: 2.4500 | Train Acc: 0.9925 | Test Acc: 0.7440 | Train F1: 0

KeyboardInterrupt: 

#### Storing Results

In [ ]:
df_epochs = pd.DataFrame(metrics_list);
df_epochs.to_csv(os.path.join(result_dir,"result.csv"),index=False);